# Setup

In [1]:
import numpy as np
import control

np.set_printoptions(precision=4)


# Lista 2

## Condições do problema

In [2]:
# Modelo da Planta
# x_ = A @ x + B @ d

A = np.array([[-4.1172, 0.7781, 0.0], [-33.8836, -3.5729, 0.0], [0.0, 1.0, 0.0]])

B = np.array([[0.5435, -39.0847, 0.0]]).T

x0 = np.array([[0.5, 0, -0.1]]).T

print("A:")
print(A)
print()

print("B:")
print(B)
print()

print("x0:")
print(x0)


A:
[[ -4.1172   0.7781   0.    ]
 [-33.8836  -3.5729   0.    ]
 [  0.       1.       0.    ]]

B:
[[  0.5435]
 [-39.0847]
 [  0.    ]]

x0:
[[ 0.5]
 [ 0. ]
 [-0.1]]


## Quais  são  os  autovalores  de  A?  O  sistema  é  estável,  instável  ou marginalmente estável? Subamortecido ou superamortecido?

In [3]:
# Autovalores de A
eigvals = np.linalg.eigvals(A)
print("polos:")
print(np.reshape(eigvals, (3, 1)))


polos:
[[ 0.    +0.j    ]
 [-3.8451+5.1275j]
 [-3.8451-5.1275j]]


Sistema marginalmente estável (2 polos no semiplano negativo e um polo na origem);

Sistema subamortecido (par de polos complexos).

## O sistema é controlável ou não? Justifique.

In [4]:
# Matriz de controlabilidade
R = np.hstack([B, A @ B, A @ A @ B])
print(np.linalg.matrix_rank(R) == A.shape[1])


True


Sistema completamente controlável, pois o rank da matriz de controlabilidade é igual ao número de estados.

## Modele o sistema dinâmico no Simulink (MATLAB), Xcos (Scilab), ou similares

Diagrama do sistema (em verde) e sistema de simulação implementado no software xcos (scilab). Em todos os modelos desse documento, os valores do projeto foram calculados utilizando o código desse documento e copiados para o modelo no software xcos conforme necessário.

![](./lista_2/diagramas/Planta.svg)

## Simule o sistema nos seguintes casos

### Entrada Nula

A imagem abaixo apresenta a simulação da resposta no tempo do sistema para entrada nula.

Nesse e em todos os próximos gráficos de resposta no tempo nesse documento, a linha laranja no gráfico superior representa a entrada na planta $\delta$ (chamada de d no gráfico por limitações do software), enquanto as linhas verde, azul e ciano, respectivamente, representam os estados $\alpha$, $q$, e $\theta$ no gráfico de baixo. O eixo horizontal em todos os gráficos vai de 0 a 10 segundos, enquanto o eixo vertical foi ajustado automaticamente para destacar os detalhes do gráfico. Comparações de magnitude dever ser feitas com observação cuidadosa da escala de cada gráfico

![](./lista_2/graficos/4a.svg)

### Entrada degrau de 0.1 rad

![](./lista_2/graficos/4b.svg)

## Projete um regulador via alocação de polos (ou seja, calcule o vetor 𝑲) que forneça um sistema com as seguintes características de malha fechada
- Sobressinal de 20%
- Tempo de pico de 1 segundo

In [5]:
# Escolha de polos
tp = 1
PO = 0.20

wd = np.pi * tp
sigma = -abs(wd * np.log(0.2) / np.pi)

print(f"wd: {wd}")
print(f"sigma: {sigma}")


wd: 3.141592653589793
sigma: -1.6094379124341


In [6]:
# Ganho do feedback completo
K = control.place(A, B, (sigma + wd * 1j, sigma - wd * 1j, 10 * sigma))
print("K:")
print(K)


K:
[[ 2.3849 -0.2642 -1.1182]]


In [7]:
# Autovalores resultantes (conferir alocação)
eigvals = np.linalg.eigvals(A - B @ K)
print("polos:")
print(np.reshape(eigvals, (3, 1)))


polos:
[[-16.0944+0.j    ]
 [ -1.6094+3.1416j]
 [ -1.6094-3.1416j]]


## Implemente a alocação de polos no Simulink (MATLAB), Xcos (Scilab), ou similares. Assuma  inicialmente  que  o  controlador  tem  acesso  ao  vetor  de  estados  completo.  Inclua imagem(ns) do controlador construído.

Diagrama do controlador (em amarelo) conectado à planta (em verde)

![](./lista_2/diagramas/FSF.svg)

### Simule o sistema com entrada nula

![](./lista_2/graficos/6a.svg)

### Simule o sistema com entrada degrau de 0.1 rad

![](./lista_2/graficos/6b.svg)

O valor de sobressinal e o tempo de pico desejados não podem ser observados nos gráficos, pois as condições iniciais dominam o sistema durante o início da resposta transiente (os valores iniciais são muito maiores que a entrada). O sistema também manteve a estabilidade de $\alpha$ e $q$, mas agora com $\theta$ estável, não mais marginalmente estável. uma entrada degrau agora desloca $\theta$, não $q$ e $\alpha$, para um degrau;

## Altere controlador para que se transforme em um rastreador com erro ao degrau nulo  em  regime  permanente.  O  sinal  de  entrada  define  o  ângulo  de  arfagem  𝜃  desejado.

O modelo de rastreador é uma modificação simples do modelo anterior, adicionando apenas um ganho proporcional $G$ à entrada de referência $r$. O diagrama completo pode ser visto na imagem abaixo, enquanto o cálculo do ganho é realizado em código python.

![](./lista_2/diagramas/fsf_g.svg)

In [8]:
# Função para calcular o ganho proporcional
def gain_scale(k, a, b, c):
    """
    Calcula o ganho proporcional da referência para um rastreador com feedback completo

    Modelo:
    dx = Ax + Bu
    y  = Cx
    u = Gr - Kx

    Método:
    https://www.control.utoronto.ca/people/profs/kwong/ece410/2008/notes/chap5.pdf
    G = [K I] * [[A B], [C 0]]^(-1) * [0 I]'
    """

    o = np.zeros_like(c).T
    i = np.eye(c.shape[0])

    m1 = np.block([k, i])
    m2 = np.linalg.inv(np.block([[a, b], [c, 0 * i]]))
    m3 = np.block([[o], [i]])

    g = m1 @ m2 @ m3
    return g


In [9]:
# Encontra o ganho proporcional
C = np.array([[0, 0, 1]])  # Rastreia theta
G = gain_scale(K, A, B, np.array([[0, 0, 1]]))
print("G:")
print(G)


G:
[[-1.1182]]


### Simule o sistema com entrada nula

![](./lista_2/graficos/7a.svg)

### Simule o sistema com degrau de 0.1 rad

![](./lista_2/graficos/7b.svg)

Conforme desejado, o estado $\theta$ agora rastreia a entrada $r$ com erro nulo ao degrau

## Projetar e simular dois reguladores diferentes projetados via LQR, listados abaixo.

O modelo do controlador LQR é o mesmo do controlador por alocação de polos desenvolvido acima mudando apenas o método de escolha do ganho/polos.

### Estados e sinais de controle com pesos iguais

In [10]:
# LQR com pesos iguais
Q1 = np.eye(3)
R1 = np.array([[1]])
K1, _, eig1 = control.lqr(A, B, Q1, R1)
# Calculado também o ganho proporcional, não necessário para a entrada nula
G1 = gain_scale(K1, A, B, C)

print("Q1:")
print(Q1)
print()

print("R1:")
print(R1)
print()

print("K1:")
print(K1)
print()

print("G1:")
print(G1)
print()

print("polos:")
print(np.reshape(eig1, (3, 1)))


Q1:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

R1:
[[1]]

K1:
[[ 0.6423 -0.9274 -1.    ]]

G1:
[[-1.]]

polos:
[[-38.4769+0.j]
 [ -4.8481+0.j]
 [ -0.9614+0.j]]


A resposta no tempo do sistema, mostrada abaixo, mostra que há uma resposta superamortecida. O pico de uso do atuador é de aproximadamente -0.46 rad.

![](./lista_2/graficos/8a.svg)

### Um ajuste que poupa (reduz) o uso do atuador (profundor)

In [11]:
# LQR com pesos variáveis, focando em baixo uso de atuador
# Optado por diminuir também a importância dos estados não rastreados
Q2 = np.array([[0.1, 0, 0], [0, 0.1, 0], [0, 0, 1]])
R2 = np.array([[10]])
K2, _, eig2 = control.lqr(A, B, Q2, R2)
G2 = gain_scale(K2, A, B, C)

print("Q2:")
print(Q2)
print()

print("R2:")
print(R2)
print()

print("K2:")
print(K2)
print()

print("G2:")
print(G2)
print()

print("polos:")
print(np.reshape(eig2, (3, 1)))


Q2:
[[0.1 0.  0. ]
 [0.  0.1 0. ]
 [0.  0.  1. ]]

R2:
[[10]]

K2:
[[ 0.2573 -0.0656 -0.3162]]

G2:
[[-0.3162]]

polos:
[[-4.5892+5.066j]
 [-4.5892-5.066j]
 [-1.2137+0.j   ]]


Conforme desejado, o segundo controlador apresenta menor uso do atuador (pico sai de -0.46 a -0.16). Entretanto, esse baixo uso do atuador tem como consequência adversa uma piora do comportamento transiente do sistema. $\theta$ e $q$ iniciamnete se distanciam do valor estável, devido a forças da dinâmica passiva da planta superiores à força do atuador ($\alpha$ alto gera momento alto).

![](./lista_2/graficos/8b.svg)

# Lista 3

## Determine a matriz de saída C para cada medida a seguir:

### Ângulo de arfagem 𝜃

In [12]:
# y = theta
C_theta = np.array([[0, 0, 1]])
print("C_theta:")
print(C_theta)


C_theta:
[[0 0 1]]


### Ângulo  de  ataque 𝛼

In [13]:
# y = alpha
C_alpha = np.array([[1, 0, 0]])
print("C_alpha:")
print(C_alpha)


C_alpha:
[[1 0 0]]


### Direção de voo 𝛾

In [14]:
# y = gamma = theta - alpha
C_gamma = np.array([[-1, 0, 1]])
print("C_gamma:")
print(C_gamma)


C_gamma:
[[-1  0  1]]


## Avalie a observabilidade para cada um dos sensores separadamente.

In [16]:
# Vetoriza a operação
C_array = np.array([C_theta, C_alpha, C_gamma])
# Calcula matrizes de observabilidade
obs = np.block([[C_array], [c := C_array @ A], [c := c @ A]])
# Verifica observabilidade
# (posto da matriz de observabilidade igual a número de estados)
print(np.linalg.matrix_rank(obs) == np.shape(A)[0])


[ True False  True]


O sistema é observável para medições de ângulo de arfagem ou de direção do voo, mas não é plenamente observável com apenas medição do ângulo de ataque

## Projete  um  observador  de  estados de  ordem  plena que  possua  um  tempo  de acomodação compatível com o controlador projetado na **Questão 7**. Considere que a saída do sistema é o ângulo de arfagem 𝜃.

In [40]:
# dx_ = A @ x_ + B @ d + H @ (y - y_)
# y_ = C * x_
# x0_ = 0

C = C_theta
# polos convergindo de 2 vezes mais rápido que controlador
polos_observador = np.linalg.eigvals(A - B @ K).real * 2
# Ganho do observador
H = control.acker(A.T, C.T, polos_observador).T

print("H:")
print(H.T)

H:
[[ 20.1995 -61.3949  30.9364]]


## Modele o observador de estados no Simulink. Altere o controlador da Questão 7 para utilizar as informações provindas do observador de estados. Mostre imagem(ns) do sistema com planta e observador de estados. Assuma que as condições iniciais do observador são nulas (o  observador  não  conheceo  estado  inicial  da  planta).  Inclua  imagem(ns)  do  observador construído.

## Simule o sistema nos seguintes casos:

> Mostrar gráficos contendo 
> - o vetor de estados (ou seja, dos 3 estados)
> - um gráfico com sinal de controle 𝛿.
> - 𝑦(𝑡)e 𝑦̂(𝑡) em um mesmo gráfico
> - 𝑦(𝑡)−𝑦̂(𝑡)
> 
> **Discuta os resultados**

### Entrada nula

### Entrada degrau de 0.1 rad

## Altere o controlador para incluir um canal integral. Simule o sistema nos seguintes casos:

> Mostrar gráficos contendo 
> - o vetor de estados (ou seja, dos 3 estados)
> - um gráfico com sinal de controle 𝛿.
> - 𝑦(𝑡)e 𝑦̂(𝑡) em um mesmo gráfico
> - 𝑦(𝑡)−𝑦̂(𝑡)
> 
> **Discuta os resultados**


In [41]:
n_out = C.shape[0]
n_x = C.shape[1]
n_in = B.shape[1]

Aa = np.block([[A, np.zeros((n_x, n_out))], [-C, np.zeros((n_out, n_out))]])
print("Aa:")
print(Aa)
print()


Ba = np.block([[B], [np.zeros((n_out, n_in))]])
print("Ba:")
print(Ba)
print()


Ca = np.block([C, np.zeros((n_out, n_out))])
print("Ca:")
print(Ca)
print()


Aa:
[[ -4.1172   0.7781   0.       0.    ]
 [-33.8836  -3.5729   0.       0.    ]
 [  0.       1.       0.       0.    ]
 [  0.       0.      -1.       0.    ]]

Ba:
[[  0.5435]
 [-39.0847]
 [  0.    ]
 [  0.    ]]

Ca:
[[0. 0. 1. 0.]]



In [42]:
# Aloca polos no sistema extendido
polos_a = np.block([np.linalg.eigvals(A - B @ K), -11 * sigma])
Ka = control.place(Aa, Ba, polos_a)

print("Ka:")
print(Ka)
print()
print(polos_a)


Ka:
[[  8.2419   0.2702   0.9115 -19.7966]]

[-16.0944+0.j      -1.6094+3.1416j  -1.6094-3.1416j  17.7038+0.j    ]


### Entrada nula

### Entrada degrau de 0.1 rad